## Example: The Tiger Problem as a Markov Decision Problem (MDP)

<center>
    <img src="figs/Fig-Linear-MDP-Schematic.png" style="align:right; width:80%">
</center>

An agent trapped in a long hallway with two doors at either end. Behind the red door is a tiger (and certain death), while behind the green door is freedom. If the agent opens the red door, the agent is eaten (and receives a large negative reward). However, if the agent opens the green door, it escapes and gets a positive reward. 

For this problem, the MDP has the tuple components:
* $\mathcal{S} = \left\{1,2,\dots,N\right\}$ while the action set is $\mathcal{A} = \left\{a_{1},a_{2}\right\}$; action $a_{1}$ moves the agent one state to the right, action $a_{2}$ moves the agent one state to the left.
* The agent receives a reward of +10 for entering state 1 (escapes). However, the agent is penalized -100 for entering state N (eaten by the tiger).  Finally, the agent is not charged to move to adjacent locations.
* Let the probability of correctly executing the action $a_{j}$ be $\alpha$

Let's compute $U^{\pi}(s)$ for different choices for the policy function $\pi$.

## Setup
Let's load some packages that are required for the example by calling the `include(...)` function on our initialization file `Include.jl`:

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5760-Examples-F23`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
   Installed PDMats ───────────── v0.11.28
   Installed TranscodingStreams ─ v0.10.0
   Installed JLD2 ─────────────── v0.4.36
    Updating `~/Desktop/julia_work/CHEME-5760-Examples-F23/Project.toml`
  [033835bb] ↑ JLD2 v0.4.35 ⇒ v0.4.36
    Updating `~/Desktop/julia_work/CHEME-5760-Examples-F23/Manifest.toml`
  [523fee87] ↑ CodecBzip2 v0.8.0 ⇒ v0.8.1
  [944b1d66] ↑ CodecZlib v0.7.2 ⇒ v0.7.3
  [033835bb] ↑ JLD2 v0.4.35 ⇒ v0.4.36
  [90014a1f] ↑ PDMats v0.11.26 ⇒ v0.11.28
  [3bb67fe8] ↑ TranscodingStreams v0.9.13 ⇒ v0.10.

In [2]:
# setup some global constants -
α = 0.75; # probability of moving the direction we are expect
γ = 0.95; # discount factor

## States and actions
Let's setup the states $\mathcal{S}$ and actions $\mathcal{A}$ sets: 
* We have `safety` at index `1`, but the `tiger` lives at index `10`. Thus, the states set $\mathcal{S} = \left\{1,\dotsc,10\right\}$
* We have `3` possible actions, `move left`, `move right` or `stand still` in the action set $\mathcal{A} = \left\{1,2,3\right\}$

In [3]:
# setup the states and actions -
safety = 1;
tiger = 10;

# Setup the states -
states = range(safety,stop=tiger, step=1) |> collect;

# Setup the actions
actions = [1,2,3]; # a₁ = move left, a₂ = move right, a₃ = stand still

## Rewards

The rewards $\mathbf{R}$ is a $\dim\mathcal{S}\times\dim\mathcal{A}$ array whose $R_{sa}$ element holds the reward for performing action $a$ in state $s$:

In [15]:
# setup the rewards -
R = Array{Float64,2}(undef,length(states), length(actions));
fill!(R,0.0) # fill R w/-1

# set the rewards for the ends -
R[safety + 1, 1] = 10;
R[tiger-1,  2] = -100;
R[2:end-1, 3] .= -1;

In [16]:
R

10×3 Matrix{Float64}:
  0.0     0.0   0.0
 10.0     0.0  -1.0
  0.0     0.0  -1.0
  0.0     0.0  -1.0
  0.0     0.0  -1.0
  0.0     0.0  -1.0
  0.0     0.0  -1.0
  0.0     0.0  -1.0
  0.0  -100.0  -1.0
  0.0     0.0   0.0

## Transitions
The transition probability array $\mathbf{T}$ is a $\dim\mathcal{S}\times\dim\mathcal{S}\times\dim\mathcal{A}$ array that encodes the physics of the world. 
* For an action $a$, the array $\mathbf{T}_{a}$ is a $\dim\mathcal{S}\times\dim\mathcal{S}$ array encoding states $s$ on the rows, and states $s^{\prime}$ on the columns. Because the entries are probabilities, the rows of $\mathbf{T}_{a}$ must sum to `1`.

In [6]:
# Setup the transitions
T = Array{Float64,3}(undef, length(states), length(states), length(actions));
fill!(T,0.0);

# We need to put values into the transition array (these are probabilities, so eah row much sum to 1)
T[safety, 1, 1:length(actions)] .= 1.0; # if we are in state 1, we stay in state 1 ∀a ∈ 𝒜
T[tiger, tiger, 1:length(actions)] .= 1.0; # if we are in state 5, we stay in state 5 

### Left, Right, and Listen Actions
We encode the probability of reaching the next state $s^{\prime}\leftarrow(s,a)$ in the entries of $\mathbf{T}_{a}$:

In [7]:
# left actions -
for s ∈ 2:(tiger - 1)
    T[s,s-1,1] = α;
    T[s,s+1,1] = (1-α);
end

# right actions -
for s ∈ 2:(tiger - 1)
    T[s,s-1,2] = (1-α);
    T[s,s+1,2] = α; 
end

# listen action (we don't move to a new state)
for s ∈ 2:(tiger-1)
    T[s,s,3] = 1.0;
end

In [21]:
T[:,:,1] # probability matrix for taking action aᵢ

10×10 Matrix{Float64}:
 1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
 0.75  0.0   0.25  0.0   0.0   0.0   0.0   0.0   0.0   0.0
 0.0   0.75  0.0   0.25  0.0   0.0   0.0   0.0   0.0   0.0
 0.0   0.0   0.75  0.0   0.25  0.0   0.0   0.0   0.0   0.0
 0.0   0.0   0.0   0.75  0.0   0.25  0.0   0.0   0.0   0.0
 0.0   0.0   0.0   0.0   0.75  0.0   0.25  0.0   0.0   0.0
 0.0   0.0   0.0   0.0   0.0   0.75  0.0   0.25  0.0   0.0
 0.0   0.0   0.0   0.0   0.0   0.0   0.75  0.0   0.25  0.0
 0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.75  0.0   0.25
 0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0

## Build the MDP problem object and estimate the utility $U^{\pi}(s)$ 
First, construct a `MyMDPProblemModel` instance with values associated with the problem. 
* We must pass the states `𝒮`, the actions `𝒜`, the transition matrix `T`, the reward matrix `R`, and the discount factor `γ` into the `build(...)` method. 

We store the MDP model in the `m` variable:

In [17]:
m = build(MyMDPProblemModel, 
    (𝒮 = states, 𝒜 = actions, T = T, R = R, γ = γ));

Next, let's formulate some simple policies for this problem, `always_move_right`, `always_move_left` and `always_listen`:

In [22]:
always_move_right(s) = 2;
always_move_left(s) = 1;
always_listen(s) = 3;

### Policy evaluation
We compute a policy’s value function (utility) using the `iterative_policy_evaluation` function, which evaluates the _immediate benefit_ of implementing the policy $\pi(s)$ and the _future benefit_ of looking ahead over states. 

* The `iterative_policy_evaluation(...)` function improves our estimate of the value (utility) of a policy $U^{\pi}(s)$ by iteration. The `iterative_policy_evaluation` function takes a `MyMDPProblemModel` instance, a policy function, and the maximum number of iterations to refine the value estimate.

```julia
function iterative_policy_evaluation(p::MyMDPProblemModel, policy::Function, k_max::Int)

    # grab stuff from the problem -
    𝒮, T, R, γ = p.𝒮, p.T, p.R, p.γ;

    # initialize U vector
    U = [0.0 for s ∈ 𝒮];

    # solve -
    for _ ∈ 1:k_max
        U = [lookahead(p, U, s, policy(s)) for s ∈ 𝒮]
    end

    # return U -
    return U;
end
```

The `lookahead(...)` function computes the immediate and future benefit. The `lookahead` function takes the `MyMDPProblemModel` instance, the value vector `U`, the state `s`, and the action `a` as arguments and returns the total value of the policy (immediate plus future benefit):

```julia
function lookahead(p::MyMDPProblemModel, U::Vector{Float64}, s::Int64, a::Int64)
    
    # get data from the problem
    𝒮, T, R, γ = p.𝒮, p.T, p.R, p.γ;
    
    # compute -
    return R[s,a] + γ*sum(T[s, s′,a]*U[i] for (i,s′) in enumerate(𝒮))
end
```

In [26]:
U = iterative_policy_evaluation(m, always_move_right, 50*length(states));

### Estimate the Action-value or Q-function
The quantity being computed in the `lookahead(...)` function is called the _action-value_ or _Q-function_ $Q(s,a)$. From $Q(s,a)$ we can compute the value function $U(s)$ by selecting the action $a$ that maximizes the _Q-function_:
\begin{equation*}
U(s) = \underset{a\in\mathcal{A}}{\max}\,Q(s,a)
\end{equation*}
and the policy $\pi(s)$::
\begin{equation*}
\pi(s) = \underset{a\in\mathcal{A}}{\arg\max}\,Q(s,a)
\end{equation*}

#### Implementation
* We compute the `Q-function` by calling the `Q(...)` function:

```julia
function Q(p::MyMDPProblemModel, U::Array{Float64,1})::Array{Float64,2}

    # grab stuff from the problem
    𝒮, T, R, γ = p.𝒮, p.T, p.R, p.γ;

    # initialize -
    Q_array = Array{Float64,2}(undef, length(𝒮), length(𝒜))

    for s ∈ 1:length(𝒮)
        for a ∈ 1:length(𝒜)
            Q_array[s,a] = R[s,a] + γ*sum([T[s, s′,a]*U[s′] for s′ in 𝒮]);
        end
    end

    return Q_array
end
```

* We estimate the best policy using the `policy(...)` function:

```julia
function policy(Q_array::Array{Float64,2})::Array{Int64,1}

    # get the dimension -
    (NR, _) = size(Q_array);

    # initialize some storage -
    π_array = Array{Int64,1}(undef, NR)
    for s ∈ 1:NR
        π_array[s] = argmax(Q_array[s,:]);
    end

    # return -
    return π_array;
end
```

In [28]:
Q_array = Q(m, U)[2:end-1,:]

8×3 Matrix{Float64}:
   -5.74413   -47.2324   -45.8708
  -52.0109    -66.2911   -63.9765
  -67.7497    -77.296    -74.4312
  -77.7503    -86.3886   -83.0691
  -86.54      -95.4818   -91.7077
  -95.5429   -105.213   -100.953
 -105.249    -115.841   -111.049
  -82.5364   -127.512   -122.137

In [14]:
best_policy = policy(Q_array)

8-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1